Author:
        
        PARK, JunHo, junho@ccnets.org

        
        KIM, JeongYoong, jeongyoong@ccnets.org
        
    COPYRIGHT (c) 2024. CCNets. All Rights reserved.

# Time Series Air Quality Data of India (2010-2023)

https://www.kaggle.com/datasets/abhisheksjha/time-series-air-quality-data-of-india-2010-2023/

*This notebook's data preprocessing is based on [here](https://www.kaggle.com/code/sotiristzamaras/india-s-air-quality-eda-ensemble-forecasting-pt-1).*

<hr>

CCNet result: https://wandb.ai/ccnets/causal-learning

<blockquote>

RMSE: <mark>0.285</mark> (Lower is better)

</blockquote>

Benchmark: https://www.kaggle.com/datasets/shibumohapatra/house-price/code?datasetId=2438863&sortBy=voteCount

<blockquote>
RMSE:

- RandomForestRegressor: 0.60
- XGBoost: 0.58

</blockquote>

<a id="1"></a>
> <h1 style = 'font-family: Times New Roman'><b> <b style = 'color: #42c2f5'>1.</b> Import Necessary Libraries </b></h1>

In [1]:
import os
import sys
import time
import glob
import warnings
warnings.filterwarnings("ignore")

import numpy as np   
import pandas as pd  

path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
dataset_name = "Time Series Air Quality Data of India"

DATASET_SRC = path_append + f'../data/Time Series Air Quality Data of India (2010-2023)'

df_states = pd.read_csv(f'{DATASET_SRC}/stations_info.csv')
df_states.drop(columns=['agency', 'station_location', 'start_month'], inplace=True)
df_states.head()

,file_name,state,city,start_month_num,start_year
0,AP001,Andhra Pradesh,Tirupati,7,2016
1,AP002,Andhra Pradesh,Vijayawada,5,2017
2,AP003,Andhra Pradesh,Visakhapatnam,7,2017
3,AP004,Andhra Pradesh,Rajamahendravaram,9,2017
4,AP005,Andhra Pradesh,Amaravati,11,2017


In [3]:
unique_states = df_states['state'].unique()
unique_states

array(['Andhra Pradesh', 'Arunachal Pradesh', 'Assam', 'Bihar',
       'Chhattisgarh', 'Chandigarh', 'Delhi', 'Gujarat',
       'Himachal Pradesh', 'Haryana', 'Jharkhand', 'Jammu and Kashmir',
       'Karnataka', 'Kerala', 'Maharashtra', 'Meghalaya', 'Manipur',
       'Madhya Pradesh', 'Mizoram', 'Nagaland', 'Odisha', 'Punjab',
       'Puducherry', 'Rajasthan', 'Sikkim', 'Telangana', 'Tamil Nadu',
       'Tripura', 'Uttarakhand', 'Uttar Pradesh', 'West Bengal'],
      dtype=object)

In [4]:
def combine_state_df(state_name):
    '''
    Combine all state files into a single dataframe and attaching the city information.

    Parameters
    ----------
        state_name (str): The name of the state

    Return
    ------
        df (DataFrame): The combined dataframe from all files of a specific state
    '''
    
    state_code = df_states[df_states['state'] == state_name]['file_name'].iloc[0][:2]
    state_files = glob.glob(f'{DATASET_SRC}/{state_code}*.csv')
    print(f'Combining a total of {len(state_files)} files...\n')

    combined_df = []
    
    for state_file in state_files:
        file_name = state_file.split(f'{DATASET_SRC}')[1][1:-4]
        file_df = pd.read_csv(state_file)
        file_df['city'] = df_states[df_states['file_name'] == file_name]['city'].values[0]
        file_df['city'] = file_df['city'].astype('string')
        combined_df.append(file_df)
        
    return pd.concat(combined_df)

In [5]:
df = combine_state_df('Delhi')

Combining a total of 40 files...



In [6]:
# Make the 'From Date' column the index as datetime
def create_dt_index(dataframe):
    dataframe = dataframe.drop(columns='To Date')
    dataframe['From Date'] = pd.to_datetime(dataframe['From Date'])
    dataframe = dataframe.rename(columns={'From Date': 'datetime'})
    return dataframe.set_index('datetime')

df = create_dt_index(df)

In [7]:
reduction_groups = {
    "Xylene (ug/m3)":    ["Xylene ()"],
    "MP-Xylene (ug/m3)": ["MP-Xylene ()"],
    "Benzene (ug/m3)":   ["Benzene ()"],
    "Toluene (ug/m3)":   ["Toluene ()"],
    "SO2 (ug/m3)":       ["SO2 ()"],
    "NOx (ug/m3)":       ["NOx (ppb)"],
    "Ozone (ug/m3)":     ["Ozone (ppb)"],
    "AT (degree C)":     ["AT ()"],
    "WD (degree)":       ["WD (degree C)", "WD (deg)", "WD ()"],
    "WS (m/s)":          ["WS ()"]
}

In [8]:
def merge_columns(dataframe, columns):
    '''
    Merges column records into a single column.

    Parameters
    ----------
        dataframe (DataFrame): The DataFrame to edit
        column (str): The name of the column to merge records into
        cols_to_merge (list[str]): A list of column names to retrieve records
    '''
    
    for column, cols_to_merge in columns.items():
        # Check if the original column exist, otherwise create it
        if column not in dataframe.columns and any(name in dataframe.columns for name in cols_to_merge):
            dataframe[column] = np.nan

        for col_name in cols_to_merge:
            if col_name in dataframe.columns:
                dataframe[column] = dataframe[column].fillna(dataframe[col_name])
                dataframe = dataframe.drop(columns=[col_name])
            
    return dataframe
df = merge_columns(df, reduction_groups)

In [9]:
df = df.dropna(how='all')
df = df.dropna(how='all', axis='columns')

In [10]:
# Threshold value indicating how much of the dataset needs to be not missing.
threshold = 0.6
df = df.dropna(thresh=df.shape[0]*threshold, axis=1)

In [11]:
df = df.resample('60min').mean(numeric_only=True)

In [12]:
df['PM2.5 (ug/m3)'] = df['PM2.5 (ug/m3)'].mask(df['PM2.5 (ug/m3)'].gt(950))
df['CO (mg/m3)'] = df['CO (mg/m3)'].mask(((df.index > '2015') & df['CO (mg/m3)'].gt(35)))
df['Ozone (ug/m3)'] = df['Ozone (ug/m3)'].mask(df['Ozone (ug/m3)'].gt(185))
df['NOx (ug/m3)'] = df['NOx (ug/m3)'].mask((
    ((df.index < '2013') & (df['NOx (ug/m3)'].gt(380))) |
    ((df.index > '2015') & (df.index < '2016') & (df['NOx (ug/m3)'].gt(400))) |
    ((df.index > '2016') & (df['NOx (ug/m3)'].gt(450)))
))
df = df.interpolate(method='pad')
df = df.fillna(df.mean())

In [13]:
date_time = pd.to_datetime(df.index, format='%d.%m.%Y %H:%M:%S')
timestamp = date_time.map(pd.Timestamp.timestamp)
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp * (2 * np.pi / year))

DATETIME_FEATURES = ['Day sin', 'Day cos', 'Year sin', 'Year cos']

In [14]:
df = df.query('datetime > 2014')

In [15]:
df.head()

,PM2.5 (ug/m3),NO (ug/m3),NO2 (ug/m3),CO (mg/m3),Ozone (ug/m3),NOx (ug/m3),Day sin,Day cos,Year sin,Year cos
datetime,,,,,,,,,,
2014-01-01 01:00:00,78.525000,16.984000,24.036667,9.9420,12.040000,46.524000,0.258819,0.965926,0.006394,0.999980
2014-01-01 02:00:00,104.290000,22.681667,25.710000,3.7475,9.910000,49.303333,0.500000,0.866025,0.007110,0.999975
2014-01-01 03:00:00,144.743333,17.354000,28.606000,11.3025,4.265000,49.328000,0.707107,0.707107,0.007827,0.999969
2014-01-01 04:00:00,147.760000,8.895000,30.920000,19.7825,5.396667,41.965000,0.866025,0.500000,0.008544,0.999963
2014-01-01 05:00:00,171.950000,11.555000,28.927500,9.3200,5.466667,41.580000,0.965926,0.258819,0.009261,0.999957


In [16]:
import torch
from sklearn.model_selection import train_test_split
from preprocessing.dataset import TemplateDataset

train_df, testset = train_test_split(df, test_size=0.2, shuffle=False)
train_df, val_df = train_test_split(train_df, test_size=0.2, shuffle=False)
# predict the next value in the sequence
train_df_x = train_df.iloc[:, :-1] # all columns except the last one
train_df_y = train_df.iloc[:, -1:] # only the last column

val_df_x = val_df.iloc[:, :-1] # all columns except the last one
val_df_y = val_df.iloc[:, -1:] # only the last column

print('train df shape: ', train_df.shape)
print('test df shape: ', val_df.shape)
min_seq_len = 16    
max_seq_len = 32
trainset = TemplateDataset(train_df_x, train_df_y, min_seq_len = min_seq_len, max_seq_len = max_seq_len)
valset = TemplateDataset(val_df_x, val_df_y, min_seq_len = max_seq_len, max_seq_len = max_seq_len)

train df shape:  (51869, 10)
test df shape:  (12968, 10)


In [17]:
from tools.config.data_config import DataConfig
from tools.config.ml_config import MLConfig
from causal_learning import CausalLearning

num_features = trainset.X.shape[1]
num_classes = trainset.y.shape[1]
data_config = DataConfig(dataset_name = dataset_name, task_type='regression', obs_shape=[num_features], label_size=num_classes)

#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_config = MLConfig(model_name = 'gpt')
ml_config.training.error_function = 'mae'
ml_config.training.num_epoch = 20

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the CausalLearning class with the training configuration, data configuration, device, and use_print and use_wandb flags
causal_learning = CausalLearning(ml_config, data_config, device, use_print=True, use_wandb=False) 

In [18]:
causal_learning.train(trainset, valset)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: junhopark (ccnets). Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (.\..\saved\time-series-air-quality-data-of-india\causal-learning)... Done. 0.0s


Trainer Name: causal_trainer


ModelConfig Parameters:


,d_model,dropout,model_name,num_layers,use_seq_input
0,256,0.05,gpt,5,True


TrainConfig Parameters:


...

[17/20][30/810][Time 9.66]
Unified LR across all optimizers: 0.0006613705467687248
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0867	Gen: 3.0699	Rec: 3.0629	E: 0.0846	R: 0.0717	P: 6.1064

mse: 0.0056
mae: 0.0520
rmse: 0.0746
r2: 0.9883

=====================Eval Metrics========================
mse: 0.1206
mae: 0.1408
rmse: 0.3473
r2: 0.7589

[17/20][130/810][Time 9.72]
Unified LR across all optimizers: 0.0006593922224215645
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0833	Gen: 3.2121	Rec: 3.2063	E: 0.0812	R: 0.0707	P: 6.4473

mse: 0.0110
mae: 0.0632
rmse: 0.1049
r2: 0.9755

=====================Eval Metrics========================
mse: 0.0678
mae: 0.1207
rmse: 0.2604
r2: 0.8510

[17/20][230/810][Time 9.57]
Unified LR across all optimizers: 0.0006574198157362082
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0936	Gen: 2.9870	Rec: 2.9805	E: 0.0911	R: 0.0790	P: 5.9929

mse: 0.0

Iterations:   0%|          | 0/810 [00:00<?, ?it/s]

[18/20][20/810][Time 9.59]
Unified LR across all optimizers: 0.0006457086589521159
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0922	Gen: 3.0668	Rec: 3.0601	E: 0.0892	R: 0.0771	P: 6.0467

mse: 0.0271
mae: 0.0532
rmse: 0.1646
r2: 0.9450

=====================Eval Metrics========================
mse: 0.0167
mae: 0.0625
rmse: 0.1294
r2: 0.9655

[18/20][120/810][Time 9.57]
Unified LR across all optimizers: 0.0006437771832197627
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0956	Gen: 2.9284	Rec: 2.9218	E: 0.0924	R: 0.0809	P: 5.8428

mse: 0.0136
mae: 0.0606
rmse: 0.1166
r2: 0.9721

=====================Eval Metrics========================
mse: 0.0495
mae: 0.0958
rmse: 0.2225
r2: 0.8981

[18/20][220/810][Time 9.64]
Unified LR across all optimizers: 0.0006418514850133148
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0881	Gen: 3.0348	Rec: 3.0286	E: 0.0866	R: 0.0752	P: 6.1008

mse: 0.0

Iterations:   0%|          | 0/810 [00:00<?, ?it/s]

[19/20][10/810][Time 9.76]
Unified LR across all optimizers: 0.0006304176596354259
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0856	Gen: 2.9536	Rec: 2.9480	E: 0.0826	R: 0.0730	P: 5.9123

mse: 0.0150
mae: 0.0569
rmse: 0.1227
r2: 0.9707

=====================Eval Metrics========================
mse: 0.0602
mae: 0.1062
rmse: 0.2454
r2: 0.8792

[19/20][110/810][Time 9.63]
Unified LR across all optimizers: 0.0006285319230978225
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0834	Gen: 2.8598	Rec: 2.8548	E: 0.0804	R: 0.0707	P: 5.7047

mse: 0.0078
mae: 0.0517
rmse: 0.0885
r2: 0.9847

=====================Eval Metrics========================
mse: 0.0929
mae: 0.1367
rmse: 0.3049
r2: 0.8114

[19/20][210/810][Time 9.56]
Unified LR across all optimizers: 0.000626651827268779
=====================Train Metrics=======================
CCNet:  Three Gpt
Inf: 0.0908	Gen: 2.9695	Rec: 2.9643	E: 0.0863	R: 0.0775	P: 5.8744

mse: 0.02

In [19]:
causal_learning.test(valset)

{'mse': 0.08158725500106812,
 'mae': 0.1333892047405243,
 'rmse': 0.2856348156929016,
 'r2': 0.8312006592750549}